In [2]:
import numpy as np
import scipy.linalg as splin
from scipy.integrate import LSODA, solve_ivp
import matplotlib.pyplot as plt

from consts import *
from lyapunov import *
from pac import *
from potential import *
from lyap_obj import *

In [ ]:
def compute_manifold(orbit: LyapOrbit, n_curves = 10):

    # Computes the stable and unstable manifolds of a given orbit
    
    # Perturbation parameter
    de = 40/384400

    # Compute the periodic orbit
    X, PHI = orbit.propagate()
    PHI = PHI.reshape(4,4)
    # Compute the length of the orbit
    length = len(X[0,:])
    
    T = orbit.T
    mu = orbit.mu
    sol_s = []
    sol_u = []

    for i in range(n_curves):
        p = X[:,i*length//n_curves]
        PHI_p = PHI[:,:,i*length//n_curves]

        assert spli.det(PHI_p) < 1e-10

         # Compute the eigenvectors and eigenvalues of the STM
        eigvals, eigvecs = splin.eig(PHI_p)

        # Compute the stable and unstable eigenvectors
        # E is stable if Re(eigval) < 1 and unstable if Re(eigval) > 1
        Es_idx = [i for i in range(len(eigvals)) if np.abs(eigvals[i]) < 1 and eigvals[i].imag == 0]
        Eu_idx = [i for i in range(len(eigvals)) if np.abs(eigvals[i]) > 1 and eigvals[i].imag == 0]

        # Take eigenvalues and eigenvectors
        Es, EVs = eigvals[Es_idx], eigvecs[:, Es_idx]
        Eu, EVu = eigvals[Eu_idx], eigvecs[:, Eu_idx]

        ### Compute deviation vectors
        # Stable
        X_s = p + de*EVs[:,0]/np.linalg.norm(EVs[:2,0])
        # Unstable
        X_u = p = de*EVu[:,0]/np.linalg.norm(EVu[:2,0])


        

        ### Propagate the deviation vectors
        # Build the initial conditions
        PHI0 = np.zeros((4,4))
        X0s = np.concatenate((X_s, PHI0.flatten()))
        X0u = np.concatenate((X_u, PHI0.flatten()))
        
        
        
        # Stable (backwards)
        sol_s = solve_ivp(fdyn, [0, -T], X0s,t_eval= np.linspace(0, -T, 1000),
                           method='LSODA', args=(T, mu), rtol=1e-12, atol=3e-12)
        # Unstable (forward)
        sol_u = solve_ivp(fdyn, [0, T],X0u, t_eval= np.linspace(0,T, 1000),
                          method='LSODA', args=(T, mu), rtol=3e-12, atol=1e-12)

        # Extract the solution
        Xs = sol_s.y[:4,:]
        Xu = sol_u.y[:4,:]

        sol_s.append(Xs)
        sol_u.append(Xu)
    # End of loop
    # Plot the manifolds
    fig, ax = plt.subplots()
    for i in range(n_curves):
        ax.plot(sol_s[i][0,:], sol_s[i][1,:], 'b')
        ax.plot(sol_u[i][0,:], sol_u[i][1,:], 'r')
    
    return sol_s, sol_u









In [6]:
# Load orbits from file
L1_0 = load_orbit("out/L1_0.npz")
L1_1 = load_orbit("out/L1_1.npz")

L2_0 = load_orbit("out/L2_0.npz")
L2_1 = load_orbit("out/L2_1.npz")

L1_orbits = [L1_0, L1_1]
L2_orbits = [L2_0, L2_1]

In [8]:
print(L1_orbits[0].Yd)

[[0.81471266]
 [0.22150514]
 [2.87071409]]


In [ ]:
# Orbits are contained in 
# orbsL1
# isoC_L2
# For good measure, set C of the orbits with a e-4 accuracy
for i in range(len(L1_orbits)):
    # Truncate to 4 decimal places without rounding
    C = np.round(L1_orbits[i].C, 4)
    L1_orbits[i].C = C
    L2_orbits[i].C = C

In [7]:
de = 40/384400

# Compute states of the orbits
pair = 0 # pair of orbits to use

# Compute the states of the orbits
# L1
X1, PHI1 = L1_orbits[pair].propagate()
# L2
X2, PHI2 = L2_orbits[pair].propagate()

T = L1_orbits[pair].Yd[2]
# Take one point in X1
p_idx = 200 # index of the point, 0 <= p_idx < len(X1)
p = X1[:4, p_idx]
PHI1_p = PHI1[:, p_idx]
PHI1_p = PHI1_p.reshape((4, 4)) 

# Compute the eigenvectors and eigenvalues of the STM at p
eigvals, eigvecs = splin.eig(PHI1_p)

# Compute the stable and unstable eigenvectors
Es_idx = [i for i in range(len(eigvals)) if np.abs(eigvals[i]) < 1 and eigvals[i].imag == 0]
Eu_idx = [i for i in range(len(eigvals)) if np.abs(eigvals[i]) > 1 and eigvals[i].imag == 0]

# Take eigenvalues and eigenvectors(only position components)
Es, EVs = eigvals[Es_idx], eigvecs[:, Es_idx]
Eu, EVu = eigvals[Eu_idx], eigvecs[:, Eu_idx]

print(f"Es: {Es}")
print(f"EVs: {EVs}")
print(f"Eu: {Eu}")
print(f"EVu: {EVu}")

Es: [ 0.01006471+0.j -0.83947859+0.j]
EVs: [[-0.33780982 -0.11763913]
 [-0.09793467  0.18194181]
 [ 0.86967445  0.27701564]
 [ 0.34635195  0.93611994]]
Eu: [99.3570566 +0.j -1.19121561+0.j]
EVu: [[ 0.27507188 -0.11475265]
 [-0.16872958 -0.01417314]
 [ 0.92912558  0.06071852]
 [-0.18053101  0.99143543]]
